<a href="https://colab.research.google.com/github/miguelgutierrez/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario_RamificacionyPodaMetaheuristica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ramificación y Poda**
Función metaheurística para calcular las cotas

In [0]:
#Valores generales

TOMAS = [
[1,1,1,1,1,0,0,0,0,0],
[0,0,1,1,1,0,0,0,0,0],
[0,1,0,0,1,0,1,0,0,0],
[1,1,0,0,0,0,1,1,0,0],
[0,1,0,1,0,0,0,1,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,0],
[1,1,0,0,0,1,0,0,1,0],
[1,1,1,0,1,0,0,1,0,0],
[1,1,1,1,0,1,0,0,0,0],
[1,0,0,1,1,0,0,0,0,0],
[1,0,1,0,0,1,0,0,0,0],
[1,1,0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0,0,1],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0],
[1,0,1,0,0,0,0,0,0,0],
[1,0,1,1,1,0,0,0,0,0],
[0,0,0,0,0,1,0,1,0,0],
[1,1,1,1,0,0,0,0,0,0],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,1],
[1,0,1,0,1,0,0,0,1,0],
[0,0,0,1,1,0,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0],
[1,0,0,0,1,1,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0]
]
TOMAS_TO_TEST = [
[1,1,1,1,1,0,0,0,0,0],
[0,0,1,1,1,0,0,0,0,0],
[0,1,0,0,1,0,1,0,0,0],
[1,1,0,0,0,0,1,1,0,0],
[0,1,0,1,0,0,0,1,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,0],
[1,1,0,0,0,1,0,0,1,0],
[1,1,1,0,1,0,0,1,0,0],
[1,1,1,1,0,1,0,0,0,0],
[1,0,0,1,1,0,0,0,0,0],
[1,0,1,0,0,1,0,0,0,0],
[1,1,0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0,0,1],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0]
]
max_tomas_dia = 6

In [0]:
#funciones generales
#toma: fila de valores 0,1 que indican si el actor en dicha posición trabaja en dicha toma
#actores de una toma: devuelve una lista con los actores 1,...,10
def actores_de_toma(toma):
  actores = []
  for index in range(len(toma)):
    if toma[index] == 1:
      actores.append(index+1)
  return actores
#solucion: tupla de sets de tomas
#devuelve el coste de la solución
def coste_de_solucion(TOMAS,solucion,coste_actor):
  coste=0
  for tomas_dia in solucion:
    coste = coste + coste_de_tomas_mismo_dia(TOMAS,tomas_dia,coste_actor)
    #print(tomas_dia)
  return coste
#tomas_dia: set de las tomas de un día (1,...,n)
#se forma un set de actores con las tomas. El total será el coste
#devuelve el coste: en este caso suponemos que todos los actores cobran lo mismo
def coste_de_tomas_mismo_dia(TOMAS,tomas_dia,coste_actor):
  actores = set({})
  for toma in tomas_dia:
    actores.update(actores_de_toma(TOMAS[toma-1]))    
  return len(actores) * coste_actor

#TEST
#print(coste_de_tomas_mismo_dia(TOMAS,{1,2,3,4,5,6},1))
#solucion = list()
#solucion.append({1,2,3,4,5,6})
#solucion.append({8, 10, 12, 13, 15, 17})
#solucion.append({7, 9, 11, 14, 16, 18})
#print(coste_de_solucion(TOMAS,solucion,1))

In [0]:
import itertools
from itertools import permutations
import math

#variables globales
mejor_solucion = list()
coste_mejor_solucion = 10e10

#devuelve un valor: basado en el coste/ahorro que supone añadir una toma ese día o dejarla para el día siguiente
#Por ejemplo: si añado una toma que tiene 3 actores: 2 que ya están trabajando y 1 que no: -> -1 (ahorro de 1 actor)
#Por ejemplo: si añado una toma que tiene 2 actores: 2 que ya están trabajando: -> -2 (ahorro de 2 actores)
#Por ejemplo: si añado una toma que tiene 2 actores: 2 que ninguno están trabajando: -> 2 (tienen que ir 2 actores)
#tomas_dia: set con las tomas
#toma: número de la toma
def valoracion_annadir_toma(TOMAS,tomas_dia,numero_toma):
  actores_trabajando = set({})
  actores_toma = actores_de_toma(TOMAS[numero_toma-1])
  for toma in tomas_dia:
    actores_trabajando.update(actores_de_toma(TOMAS[toma-1]))    

  actores_ya_trabajando = actores_trabajando.intersection(actores_toma)

  actores_tiene_ir = set(actores_toma) - actores_ya_trabajando
  return (len(actores_tiene_ir) - len(actores_ya_trabajando))

#test
#tomas_dia = {1,3}
#toma = 6
#valoracion_annadir_toma(TOMAS,tomas_dia,toma)

#A partir de las tomas_dia (len >0) Genera los hijos que tengan la valoración menor 
#max_hijos: máximo para evitar supuestas ramificaciones excesivas
def crear_hijos(TOMAS,tomas,tomas_dia,max_hijos):
  HIJOS = []
  valoracion_minima = 10e10  

  tomas_pendientes = set(tomas) - tomas_dia
  #print("tomas_pendientes:", tomas_pendientes)
  for toma in tomas_pendientes: 
    valoracion = valoracion_annadir_toma(TOMAS,tomas_dia,toma)
    if valoracion < valoracion_minima:
      valoracion_minima = valoracion

  #print("valoracion_minima: ", valoracion_minima)
  for toma in tomas_pendientes: 
    valoracion = valoracion_annadir_toma(TOMAS,tomas_dia,toma)
    if valoracion_minima == valoracion and len(HIJOS)<max_hijos:
      HIJOS.append({toma})
  return HIJOS

#test
#tomas_totales = set(range(1,len(TOMAS)+1)) 
#print(crear_hijos(TOMAS,tomas_totales,{1,6,7,20,22},5))

#quita las tomas que no tengan ningún actor en común. Es decir, tomas que todos sus actores son nuevos ese día
#en ese caso, hacemos que vayan el día siguiente. Esto es factible, sólo en el caso, que no tengamos la restricción de realizar las tomas lo antes posible
def crear_hijos_si_tienen_actores_comunes(TOMAS,tomas,tomas_dia,max_hijos):
  HIJOS = crear_hijos(TOMAS,tomas, tomas_dia,max_hijos)
  actores_trabajando = set({})
  for toma in tomas_dia:
    actores_trabajando.update(actores_de_toma(TOMAS[toma-1]))    

  for hijo in HIJOS:
    actores_hijo = actores_de_toma(TOMAS[hijo-1])
    actores_ya_trabajando = actores_trabajando.intersection(actores_hijo)
    if len(actores_ya_trabajando) == 0:
      HIJOS.remove(hijo)
  return HIJOS

#test
#tomas_totales = { 11, 21} 
#print(crear_hijos(TOMAS,tomas_totales,{16},5))
#print(crear_hijos_si_tienen_actores_comunes(TOMAS,tomas_totales,{16},5))

#se utiliza para seleccionar el primer nodo del día
#todo: investigar otras funciones
def toma_con_mas_actores(TOMAS,tomas):
  max_actores = 0 
  toma_seleccionada = 0
  for toma in tomas: 
    count_actores = len(actores_de_toma(TOMAS[toma-1]))
    if count_actores > max_actores:
      toma_seleccionada = toma
      max_actores = count_actores    
  return toma_seleccionada

#test
#tomas_totales = set(range(1,len(TOMAS)+1)) 
#print(toma_con_mas_actores(TOMAS,tomas_totales))


In [138]:
import time

#Nodos del grafo  {1,3,5} números de las tomas. He incluído un máximo para evitar ramificaciones excesivas. 
#Nodos del grafo: ({dia0,(lista dia0)})
#   lista dia0: son las posibles tomas día 0 y para cada elemento se incluye (lista dia1), es decir tenemos un par: {dia, (lista dia)}
#   y así recursivamente hasta que no se pueden generar más días
#Se debería probar a cambiar dicho número

#NODOS: array de días con los nodos posibles de ese día
NODOS=[]
dia = 0

#lista de sets, nodo es un set
def existeNodoEnLista(nodo,lista):  
  for elemento in lista:
    if elemento == nodo:
      return True

  return False

def anadir_hijos(RAMA,HIJOS_ANTERIORES,tomas_pendientes,iteracion,max_tomas_dia):  
  HIJOS = []
  for nodo in HIJOS_ANTERIORES:
    for x in crear_hijos(TOMAS,tomas_pendientes,nodo,5):
      nuevo_nodo = nodo.union(x)
      #compruebo que no esté añadido
      if not existeNodoEnLista(nuevo_nodo,HIJOS):
        HIJOS.append(nuevo_nodo)

  #print("HIJOS FINAL", HIJOS)  
  if (iteracion+1) < max_tomas_dia:    
    anadir_hijos(RAMA,HIJOS,tomas_pendientes,iteracion+1,max_tomas_dia)
  else:
    RAMA.append(HIJOS)


def ramificacion_y_poda_dia(TOMAS, tomas_pendientes):
  start_time = time.time()
  #nodo_inicial = set(toma_con_mas_actores(TOMAS, tomas_pendientes))  
  toma_mas_actores = toma_con_mas_actores(TOMAS, tomas_pendientes)
  #print("toma_mas_actores: ",toma_mas_actores)
  nodo_inicial = {toma_mas_actores}

  nodos_iniciales = []
  nodos_iniciales.append(nodo_inicial)
  rama = []
  anadir_hijos(rama,nodos_iniciales, tomas_pendientes,1,6)

  #dia1
  rama_dia_1 = calcula_ramas_dia1(rama[0],tomas_pendientes)
  #for elemento in rama_dia_1:
  #  print("elemento 1:", elemento)

  #dia2
  rama_dia_2 = calcula_ramas_dia2(rama_dia_1,tomas_pendientes)

  #dia3 
  rama_dia_3 = calcula_ramas_dia3(rama_dia_2,tomas_pendientes)

  #dia4
  rama_dia_4 = calcula_ramas_dia4(rama_dia_3,tomas_pendientes)

  #dia5, el último elemento es una lista con 1 sólo elemento
  soluciones = []
  for elemento in rama_dia_4:
    dia5 = elemento[4][0]
    soluciones.append([ elemento[0], elemento[1], elemento[2], elemento[3], dia5])

  print("Nivel complejidad. Total soluciones: ", len(soluciones))  
  solucion = calcula_mejor_solucion(TOMAS,soluciones)
  print("Duración: ", int((time.time() - start_time)), " segundos")

def calcula_ramas_dia1(rama,tomas_pendientes):
  rama_dia_1 = []
  for dia1 in rama:
    tomas_pendientes_dia1 = tomas_pendientes - dia1
    toma_mas_actores1 = toma_con_mas_actores(TOMAS, tomas_pendientes_dia1)

    nodo_inicial1 = {toma_mas_actores1}
    nodos_iniciales1 = []
    nodos_iniciales1.append(nodo_inicial1)
    rama1 = []
    anadir_hijos(rama1,nodos_iniciales1, tomas_pendientes_dia1,1,6)
    
    rama_dia_1.append([dia1,rama1[0]])

  return rama_dia_1

def calcula_ramas_dia2(rama_dia_1,tomas_pendientes):
  rama_dia_2 = []
  for elemento in rama_dia_1:
    dia1 = elemento[0]
    for dia2 in elemento[1]:
      #print("dia1:", dia1)
      tomas_pendientes_dia2 = tomas_pendientes - dia1 - dia2
      toma_mas_actores2 = toma_con_mas_actores(TOMAS, tomas_pendientes_dia2)
      nodo_inicial2 = {toma_mas_actores2}
      nodos_iniciales2 = []
      nodos_iniciales2.append(nodo_inicial2)
      rama2 = []
      anadir_hijos(rama2,nodos_iniciales2, tomas_pendientes_dia2,1,6)
      
      rama_dia_2.append([dia1,dia2,rama2[0]])

  return rama_dia_2

def calcula_ramas_dia3(rama_dia_2,tomas_pendientes):
  rama_dia_3 = []
  for elemento in rama_dia_2:
    dia1 = elemento[0]
    dia2 = elemento[1]
    for dia3 in elemento[2]:
      tomas_pendientes_dia3 = tomas_pendientes - dia1 - dia2 - dia3
      toma_mas_actores3 = toma_con_mas_actores(TOMAS, tomas_pendientes_dia3)

      nodo_inicial3 = {toma_mas_actores3}
      nodos_iniciales3 = []
      nodos_iniciales3.append(nodo_inicial3)
      rama3 = []
      anadir_hijos(rama3,nodos_iniciales3, tomas_pendientes_dia3,1,6)
      
      rama_dia_3.append([dia1,dia2,dia3, rama3[0]])

  return rama_dia_3

def calcula_ramas_dia4(rama_dia_3,tomas_pendientes):
  rama_dia_4 = []
  for elemento in rama_dia_3:
    dia1 = elemento[0]
    dia2 = elemento[1]
    dia3 = elemento[2]
    for dia4 in elemento[3]:
      tomas_pendientes_dia4 = tomas_pendientes - dia1 - dia2 - dia3 - dia4
      toma_mas_actores4 = toma_con_mas_actores(TOMAS, tomas_pendientes_dia4)

      nodo_inicial4 = {toma_mas_actores4}
      nodos_iniciales4 = []
      nodos_iniciales4.append(nodo_inicial4)
      rama4 = []
      anadir_hijos(rama4,nodos_iniciales4, tomas_pendientes_dia4,1,6)
      
      rama_dia_4.append([dia1,dia2,dia3,dia4, rama4[0]])

  return rama_dia_4

def calcula_mejor_solucion(TOMAS, soluciones):
  mejor_solucion = list()
  coste_mejor_solucion = 10e10

  for solucion in soluciones:  
    coste = coste_de_solucion(TOMAS,solucion,1)
    
    if (coste < coste_mejor_solucion):
        print("solución parcial: ", solucion)
        print("coste parcial:", coste)
        mejor_solucion = solucion 
        coste_mejor_solucion = coste         
  
  print("Mejor solucion:", mejor_solucion)
  print("Coste mejor solucion:", coste_mejor_solucion)
  

#test
tomas_totales = set(range(1,len(TOMAS)+1)) 
ramificacion_y_poda_dia(TOMAS,tomas_totales)



Nivel complejidad. Total soluciones:  19874
solución parcial:  [{1, 2, 20, 6, 7, 22}, {17, 3, 4, 19, 11, 15}, {18, 8, 9, 10, 12, 14}, {16, 5, 25, 28, 13, 30}, {21, 23, 24, 26, 27, 29}]
coste parcial: 30
solución parcial:  [{1, 2, 20, 6, 7, 22}, {17, 3, 4, 19, 11, 15}, {18, 23, 8, 9, 12, 14}, {13, 26, 24, 10, 27, 29}, {16, 5, 21, 25, 28, 30}]
coste parcial: 29
solución parcial:  [{1, 2, 20, 6, 7, 22}, {26, 5, 8, 10, 11, 12}, {3, 4, 21, 9, 29, 15}, {16, 25, 27, 28, 13, 30}, {17, 18, 19, 23, 24, 14}]
coste parcial: 28
solución parcial:  [{1, 2, 20, 6, 7, 22}, {26, 5, 9, 10, 11, 12}, {3, 4, 21, 8, 29, 15}, {16, 25, 27, 28, 13, 30}, {17, 18, 19, 23, 24, 14}]
coste parcial: 27
Mejor solucion: [{1, 2, 20, 6, 7, 22}, {26, 5, 9, 10, 11, 12}, {3, 4, 21, 8, 29, 15}, {16, 25, 27, 28, 13, 30}, {17, 18, 19, 23, 24, 14}]
Coste mejor solucion: 27
Duración:  16  segundos
